In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"veronicaastorino","key":"d70bec0f4a36fef2eb100d1b63c71351"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d stackoverflow/stacksample -p ./data

100% 1.11G/1.11G [00:43<00:00, 49.4MB/s]
100% 1.11G/1.11G [00:43<00:00, 27.1MB/s]


In [ ]:
!unzip ./data/stacksample.zip

Archive:  ./data/stacksample.zip
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz 
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 212.4MB 68kB/s 
     |████████████████████████████████| 204kB 11.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=ad7cc55564751663e3b501ee645d9600ba927171adf3fef2eca1f4405d93d530
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import findspark
findspark.init("spark-3.1.1-bin-hadoop2.7")
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext

In [4]:
df = spark.read\
  .option("multiLine", "true")\
  .option("header", "true")\
  .option("escape", "\"")\
  .csv("Questions.csv")\
  .select("Id","Body")

df.show(10)

+---+--------------------+
| Id|                Body|
+---+--------------------+
| 80|<p>I've written a...|
| 90|<p>Are there any ...|
|120|<p>Has anyone got...|
|180|<p>This is someth...|
|260|<p>I have a littl...|
|330|<p>I am working o...|
|470|<p>I've been writ...|
|580|<p>I wonder how y...|
|650|<p>I would like t...|
|810|<p>I'm trying to ...|
+---+--------------------+
only showing top 10 rows



In [ ]:
df.count()

1264216

In [6]:
from pyspark.sql.functions import col, lower, regexp_replace, split

def clean_text(c):
  c = lower(c)
  c = regexp_replace(c, "^rt ", "")
  c = regexp_replace(c, "<p>", "")
  c = regexp_replace(c, "</p>", "")
  c = regexp_replace(c, "(https?\://)\S+", "")
  c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
  return c

clean_text_df = df.select('Id',clean_text(col("Body")).alias("Body"))

In [7]:
clean_text_df.show()

+----+--------------------+
|  Id|                Body|
+----+--------------------+
|  80|ive written a dat...|
|  90|are there any rea...|
| 120|has anyone got ex...|
| 180|this is something...|
| 260|i have a little g...|
| 330|i am working on a...|
| 470|ive been writing ...|
| 580|i wonder how you ...|
| 650|i would like the ...|
| 810|im trying to main...|
| 930|whats the simples...|
|1010|i need to grab th...|
|1040|im looking for a ...|
|1070|what is the corre...|
|1160|i am using ccnet ...|
|1180|i am looking to a...|
|1300|i am using msbuil...|
|1390|im setting up a d...|
|1600|i always create a...|
|1610|if im adding a co...|
+----+--------------------+
only showing top 20 rows



In [8]:
#tokenizer text
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="Body", outputCol="vector")
vector_df = tokenizer.transform(clean_text_df)

vector_df.printSchema()
vector_df.show(10)

root
 |-- Id: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- vector: array (nullable = true)
 |    |-- element: string (containsNull = true)

+---+--------------------+--------------------+
| Id|                Body|              vector|
+---+--------------------+--------------------+
| 80|ive written a dat...|[ive, written, a,...|
| 90|are there any rea...|[are, there, any,...|
|120|has anyone got ex...|[has, anyone, got...|
|180|this is something...|[this, is, someth...|
|260|i have a little g...|[i, have, a, litt...|
|330|i am working on a...|[i, am, working, ...|
|470|ive been writing ...|[ive, been, writi...|
|580|i wonder how you ...|[i, wonder, how, ...|
|650|i would like the ...|[i, would, like, ...|
|810|im trying to main...|[im, trying, to, ...|
+---+--------------------+--------------------+
only showing top 10 rows



In [9]:
#removing stopwords
from pyspark.ml.feature import StopWordsRemover

# Define a list of stop words or use default list
remover = StopWordsRemover()
stopwords = remover.getStopWords() 

# Display default list
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

In [10]:
# Specify input/output columns
remover.setInputCol("vector")
remover.setOutputCol("vector_no_stopw")

# Transform existing dataframe with the StopWordsRemover
vector_no_stopw_df = remover.transform(vector_df)

# Display
vector_no_stopw_df.printSchema()
vector_no_stopw_df.show()

root
 |-- Id: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- vector: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vector_no_stopw: array (nullable = true)
 |    |-- element: string (containsNull = true)

+----+--------------------+--------------------+--------------------+
|  Id|                Body|              vector|     vector_no_stopw|
+----+--------------------+--------------------+--------------------+
|  80|ive written a dat...|[ive, written, a,...|[ive, written, da...|
|  90|are there any rea...|[are, there, any,...|[really, good, tu...|
| 120|has anyone got ex...|[has, anyone, got...|[anyone, got, exp...|
| 180|this is something...|[this, is, someth...|[something, ive, ...|
| 260|i have a little g...|[i, have, a, litt...|[little, game, wr...|
| 330|i am working on a...|[i, am, working, ...|[working, collect...|
| 470|ive been writing ...|[ive, been, writi...|[ive, writing, we...|
| 580|i wonder how you ...|[i, wonder, how, .

In [11]:
#To reduce the chance of collision, we can increase the target feature dimension, i.e., the number of buckets of the hash table. 
#The default feature dimension is 2^20=1,048,576. (the other model 8= 2^3)
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
hashingTF = HashingTF(inputCol="vector_no_stopw", outputCol="rawFeatures", numFeatures=100000)

featurized_data = hashingTF.transform(vector_no_stopw_df)
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurized_data)
TfIdfData = idfModel.transform(featurized_data)
TfIdfData.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  Id|                Body|              vector|     vector_no_stopw|         rawFeatures|            features|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  80|ive written a dat...|[ive, written, a,...|[ive, written, da...|(100000,[585,3835...|(100000,[585,3835...|
|  90|are there any rea...|[are, there, any,...|[really, good, tu...|(100000,[2543,162...|(100000,[2543,162...|
| 120|has anyone got ex...|[has, anyone, got...|[anyone, got, exp...|(100000,[2143,399...|(100000,[2143,399...|
| 180|this is something...|[this, is, someth...|[something, ive, ...|(100000,[6056,606...|(100000,[6056,606...|
| 260|i have a little g...|[i, have, a, litt...|[little, game, wr...|(100000,[585,2144...|(100000,[585,2144...|
| 330|i am working on a...|[i, am, working, ...|[working, collect...|(100000,[585,2869...|(100000,[585,2

In [12]:
#MinHashLSH
from pyspark.ml.feature import MinHashLSH
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)
LSHmodel = mh.fit(TfIdfData)

LSH = LSHmodel.transform(TfIdfData)
LSH.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  Id|                Body|              vector|     vector_no_stopw|         rawFeatures|            features|              hashes|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  80|ive written a dat...|[ive, written, a,...|[ive, written, da...|(100000,[585,3835...|(100000,[585,3835...|[[429673.0], [2.0...|
|  90|are there any rea...|[are, there, any,...|[really, good, tu...|(100000,[2543,162...|(100000,[2543,162...|[[7.2795738E7], [...|
| 120|has anyone got ex...|[has, anyone, got...|[anyone, got, exp...|(100000,[2143,399...|(100000,[2143,399...|[[9427095.0], [6....|
| 180|this is something...|[this, is, someth...|[something, ive, ...|(100000,[6056,606...|(100000,[6056,606...|[[2.605491E7], [2...|
| 260|i have a little g...|[i, have, a, litt...|[little, game, wr...|

In [18]:
from pyspark.ml.linalg import Vectors
key = Vectors.sparse(6, [1, 3], [1.0, 1.0])

In [19]:
LSHmodel.approxNearestNeighbors(TfIdfData, key, 10).show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|      Id|                Body|              vector|     vector_no_stopw|         rawFeatures|            features|              hashes|           distCol|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|20874280|why wont the belo...|[why, wont, the, ...|[wont, make, var,...|(100000,[3,9043,1...|(100000,[3,9043,1...|[[3.8890193E7], [...|0.9285714285714286|
|32726280|i have been runni...|[i, have, been, r...|[running, blank, ...|(100000,[3,668,41...|(100000,[3,668,41...|[[3.8890193E7], [...|0.9333333333333333|
|15190640|i am looking at s...|[i, am, looking, ...|[looking, c, code...|(100000,[1,28,573...|(100000,[1,28,573...|[[2.7236249E7], [...|0.9523809523809523|
|29467680|i want to show a ...|[i, want, to, sho...|[want, show,

In [17]:
LSH.select('vector').take(1)

[Row(vector=['ive', 'written', 'a', 'database', 'generation', 'script', 'in', 'a', 'href', 'and', 'want', 'to', 'execute', 'it', 'in', 'my', 'a', 'href', 'aira', 'application', '', 'precodecreate', 'table', 'trole', '', '', '', '', '', '', '', 'roleid', 'integer', 'primary', 'key', '', '', '', '', '', '', 'rolename', 'varchar40', '', 'create', 'table', 'tfile', '', '', '', '', '', 'fileid', 'integer', 'primary', 'key', '', '', '', '', 'filename', 'varchar50', '', '', '', '', 'filedescription', 'varchar500', '', '', '', '', 'thumbnailid', 'integer', '', '', '', '', 'fileformatid', 'integer', '', '', '', '', 'categoryid', 'integer', '', '', '', '', 'isfavorite', 'boolean', '', '', '', '', 'dateadded', 'date', '', '', '', '', 'globalaccesscount', 'integer', '', '', '', '', 'lastaccesstime', 'date', '', '', '', '', 'downloadcomplete', 'boolean', '', '', '', '', 'isnew', 'boolean', '', '', '', '', 'isspotlight', 'boolean', '', '', '', '', 'duration', 'varchar30', '', 'create', 'table', 'tca

In [20]:
similar=LSHmodel.approxNearestNeighbors(TfIdfData, key, 10)
similar.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|      Id|                Body|              vector|     vector_no_stopw|         rawFeatures|            features|              hashes|           distCol|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|20874280|why wont the belo...|[why, wont, the, ...|[wont, make, var,...|(100000,[3,9043,1...|(100000,[3,9043,1...|[[3.8890193E7], [...|0.9285714285714286|
|32726280|i have been runni...|[i, have, been, r...|[running, blank, ...|(100000,[3,668,41...|(100000,[3,668,41...|[[3.8890193E7], [...|0.9333333333333333|
|15190640|i am looking at s...|[i, am, looking, ...|[looking, c, code...|(100000,[1,28,573...|(100000,[1,28,573...|[[2.7236249E7], [...|0.9523809523809523|
|29467680|i want to show a ...|[i, want, to, sho...|[want, show,

In [21]:
sim=similar.filter((col("distCol") <= 0.60))

In [24]:
similar.select('Body').show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      